#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)])


In [0]:
driver_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                  StructField("driverRef", StringType(), True),
                                  StructField("number", IntegerType(), True),
                                  StructField("code", StringType(), True),
                                  StructField("name", name_schema),
                                  StructField("dob", DateType(), True),
                                  StructField("nationality", StringType(), True),
                                  StructField("url", StringType(), True)])


In [0]:
driver_df = spark.read \
.option("header", True)\
.schema(driver_schema) \
.json(f"{raw_folder_path}/{v_file_date}/drivers.json")

In [0]:
driver_renamed_df = driver_df \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("driverRef", "driver_ref") \
.withColumnRenamed("dob", "date_of_birth")\
.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))\
.withColumn("ingestion_date", current_timestamp())\
.withColumn("data_source", lit(v_data_source))\
.drop("url")\
.withColumn("file_date", lit(v_file_date))

In [0]:
driver_renamed_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
dbutils.notebook.exit("Success")